In [1]:
import joblib
import numpy as np
import pandas as pd
import featuretools as ft
import missingno as msno
from sklearn.model_selection import train_test_split, cross_val_score
from classification_model.config import features, target, \
                                        numerical_features, categorical_features
from classification_model.custom_pipeline import ColumnSelector, ConvertDtypes, \
                                                 GetDummies, GetDataFrame
from classification_model.evaluation import generate_report, confusion_matrix, metrics_summary
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
import warnings

pd.set_option('max_rows', 500), pd.set_option('max_columns', 500), pd.set_option('float_format', '{:.2f}'.format)
warnings.filterwarnings(action='ignore')

In [2]:
dtypes = {
    'cedula1': 'category',
    'franquicia': 'category',
    'tipo_tarjeta': 'category'
}
estado = pd.read_csv('data/estado_tc.csv', sep=';', 
                     parse_dates=['fecha_de_emision', 'fecha_ult_aumento_cupo'], dtype=dtypes)
print(f'El conjunto de datos tiene {estado.shape[0]} observaciones, y {estado.shape[1]} variables')

El conjunto de datos tiene 649133 observaciones, y 12 variables


In [3]:
estado['aumento_cupo'] = estado['fecha_ult_aumento_cupo'].apply(lambda x: 1 if x != '0' else 0)
estado['codigo_estado_tarjeta'] = estado['codigo_estado_tarjeta'].apply(lambda x: 1 if x !=0 else 0)
estado['franquicia'] = estado['franquicia'].apply(lambda x: str(x).lower())
estado['tipo_tarjeta'] = estado['tipo_tarjeta'].apply(lambda x: str(x).lower())

In [4]:
subset = ['cedula1', 'franquicia', 'tipo_tarjeta',
          'numerotarjeta1', 'fecha_de_emision', 'codigo_vendedor', 'codigo_estado_tarjeta']

In [5]:
estado = estado.drop_duplicates(subset=subset, keep='last')
estado.drop_duplicates(subset=['numerotarjeta1'], keep=False, inplace=True)
estado.drop(['fecha_ult_aumento_cupo', 'codigo_vendedor'], axis=1, inplace=True)

In [6]:
codigo_estado_tarjeta = estado.pop('codigo_estado_tarjeta')
codigo_estado_tarjeta = pd.concat([codigo_estado_tarjeta, estado['numerotarjeta1']], axis=1)

In [7]:
dtypes = {
    'cedula1': 'category',
    'codigo_transaccion': 'category'
}
trx = pd.read_csv('data/trx.csv', sep=';',
                  parse_dates=['fecha_de_transaccion'], dtype=dtypes)

In [8]:
es = ft.EntitySet(id='users')
es.entity_from_dataframe(entity_id='users',
                         dataframe=estado,
                         index='numerotarjeta1',
                         variable_types={'aumento_cupo': ft.variable_types.Categorical})

Entityset: users
  Entities:
    users [Rows: 126541, Columns: 10]
  Relationships:
    No relationships

In [9]:
es.entity_from_dataframe(entity_id='trx', 
                         dataframe=trx,
                         variable_types={'codigo_transaccion': ft.variable_types.Categorical},
                         make_index=True,
                         index='trx_id',
                         time_index='fecha_de_transaccion')

Entityset: users
  Entities:
    users [Rows: 126541, Columns: 10]
    trx [Rows: 2045721, Columns: 7]
  Relationships:
    No relationships

In [10]:
relationship = ft.Relationship(es['users']['numerotarjeta1'],
                               es['trx']['numerotarjeta1'])
es = es.add_relationship(relationship=relationship)

In [11]:
np.random.seed(seed=42)
data, features_names = ft.dfs(entityset=es, target_entity='users', max_depth=2)

data['numerotarjeta1'] = data.index
data.reset_index(drop=True, inplace=True)
data = pd.merge(left=data, right=codigo_estado_tarjeta,
                    how='left', on=['numerotarjeta1'])
print(f'El conjunto de datos tiene {data.shape[0]} observaciones, y {data.shape[1]} variables.')
data.head()

El conjunto de datos tiene 126541 observaciones, y 39 variables.


,cedula1,franquicia,tipo_tarjeta,cupo_global,disponible,total_mes_en_mora,altura_de_mora,aumento_cupo,SUM(trx.valor_transaccion),SUM(trx.numero_cuotas_diferidas),STD(trx.valor_transaccion),STD(trx.numero_cuotas_diferidas),MAX(trx.valor_transaccion),MAX(trx.numero_cuotas_diferidas),SKEW(trx.valor_transaccion),SKEW(trx.numero_cuotas_diferidas),MIN(trx.valor_transaccion),MIN(trx.numero_cuotas_diferidas),MEAN(trx.valor_transaccion),MEAN(trx.numero_cuotas_diferidas),COUNT(trx),NUM_UNIQUE(trx.cedula1),NUM_UNIQUE(trx.codigo_transaccion),MODE(trx.cedula1),MODE(trx.codigo_transaccion),DAY(fecha_de_emision),YEAR(fecha_de_emision),MONTH(fecha_de_emision),WEEKDAY(fecha_de_emision),NUM_UNIQUE(trx.DAY(fecha_de_transaccion)),NUM_UNIQUE(trx.MONTH(fecha_de_transaccion)),NUM_UNIQUE(trx.WEEKDAY(fecha_de_transaccion)),NUM_UNIQUE(trx.YEAR(fecha_de_transaccion)),MODE(trx.DAY(fecha_de_transaccion)),MODE(trx.MONTH(fecha_de_transaccion)),MODE(trx.WEEKDAY(fecha_de_transaccion)),MODE(trx.YEAR(fecha_de_transaccion)),numerotarjeta1,codigo_estado_tarjeta
0,126039,amex,azul,3000000.00,3000000.00,0,0.00,0,135511.50,182.00,10841.27,5.97,28900.00,12.00,1.78,-0.34,4.72,0.00,5211.98,7.00,26.00,1.00,4.00,126039,7,9,2014,7,2,17.00,5.00,6.00,1.00,20.00,1.00,0.00,2015.00,9941,0
1,126036,visa,clasica tradicional,3000000.00,2240678.97,9,0.00,0,2882325.43,105.00,206892.79,6.56,929164.00,24.00,3.21,1.55,15700.00,0.00,120096.89,4.38,24.00,1.00,4.00,126036,6,31,2012,7,1,14.00,5.00,7.00,1.00,22.00,5.00,4.00,2015.00,61128,0
2,126000,mastercard,mastercard platinum,10000000.00,5345069.98,0,0.00,0,303694.92,313.00,23359.03,7.16,101630.00,24.00,2.71,0.11,3.14,0.00,9202.88,9.48,33.00,1.00,4.00,126000,7,21,2011,6,1,18.00,5.00,7.00,1.00,5.00,1.00,0.00,2015.00,128985,0
3,17645,amex,azul,1200000.00,517174.73,0,0.00,0,851.23,24.00,35.26,4.67,149.05,12.00,1.68,2.06,42.00,0.00,70.94,2.00,12.00,1.00,2.00,17645,4,15,2012,5,1,8.00,10.00,5.00,1.00,25.00,3.00,2.00,2015.00,17005,0
4,64129,amex,azul,2000000.00,1036992.90,9,0.00,0,757600.00,0.00,42880.04,0.00,178000.00,0.00,1.81,0.00,28900.00,0.00,68872.73,0.00,11.00,1.00,2.00,64129,4,17,2012,5,3,7.00,10.00,5.00,1.00,29.00,9.00,4.00,2015.00,2320,0


In [12]:
cols = ['cedula1', 'NUM_UNIQUE(trx.cedula1)', 'MODE(trx.cedula1)']
data.drop(cols, axis=1, inplace=True)

In [13]:
for col in data.columns[data.isnull().sum() > 0].tolist():
    data[col] = data[col].fillna(0)

In [14]:
label = data.pop(target)
X = data.drop('numerotarjeta1', axis=1)
y = label

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
print(f'El conjunto de entrenamiento tiene {X_train.shape[0]} observaciones, y {X_train.shape[1]} variables')
print(f'El conjunto de entrenamiento tiene {X_test.shape[0]} observaciones, y {X_test.shape[1]} variables')

El conjunto de entrenamiento tiene 88578 observaciones, y 34 variables
El conjunto de entrenamiento tiene 37963 observaciones, y 34 variables


In [16]:
global_transformations = Pipeline([('selector', ColumnSelector(columns=numerical_features + categorical_features)),
                                   ('convert_dtypes', ConvertDtypes(numerical=numerical_features,
                                                                    categorical=categorical_features))])

numerical_transformations = Pipeline([('selector', ColumnSelector(columns=numerical_features)),
                                      ('scaler', StandardScaler()),
                                      ('dataframe', GetDataFrame(columns=numerical_features))])

categorical_transformations = Pipeline([('selector', ColumnSelector(columns=categorical_features)),
                                        ('ohe', GetDummies(columns=categorical_features))])

In [17]:
preprocessing = Pipeline([('globals', global_transformations),
                          ('features', FeatureUnion([
                              ('numerical', numerical_transformations),
                              ('categorical', categorical_transformations)
                          ])),
                          ('dataframe', GetDataFrame(columns=features))])

In [18]:
lr = Pipeline([('preprocessing', preprocessing),
               ('estimator', LogisticRegression(random_state=42))])
svm = Pipeline([('preprocessing', preprocessing),
                ('estimator', SVC(random_state=42))])
rf = Pipeline([('preprocessing', preprocessing),
               ('estimator', RandomForestClassifier(random_state=42))])
gb = Pipeline([('preprocessing', preprocessing),
               ('estimator', GradientBoostingClassifier(random_state=42))])

In [21]:
roc = {
    'lr': [cross_val_score(estimator=lr, X=X, y=y, scoring='roc_auc', cv=10).mean()],
    'svm': [cross_val_score(estimator=svm, X=X, y=y, scoring='roc_auc', cv=10).mean()],
    'rf': [cross_val_score(estimator=rf, X=X, y=y, scoring='roc_auc', cv=10).mean()],
    'gb': [cross_val_score(estimator=gb, X=X, y=y, scoring='roc_auc', cv=10).mean()]
       }

In [22]:
resultados = pd.DataFrame.from_dict(data=roc).T
resultados.columns = ['ROC']
resultados

,ROC
lr,0.99
svm,0.99
rf,0.99
gb,1.00


## Construcción del modelo

### Gradient Boosting

In [19]:
gb.fit(X_train, y_train)
y_pred = gb.predict(X_test)
joblib.dump(gb, filename='models/gradient_boosting.pkl')

generate_report(y_test, y_pred)

,precision,recall,f1-score,support
0,0.99,0.98,0.99,18115.00
1,0.98,0.99,0.99,19848.00
accuracy,0.99,0.99,0.99,0.99
macro avg,0.99,0.99,0.99,37963.00
weighted avg,0.99,0.99,0.99,37963.00


In [20]:
metrics_summary(y_test, y_pred)
confusion_matrix(y_test, y_pred)

El área bajo la curva ROC es: 0.9856437937432964
La exactitud es: 0.9859863551352633
La precisión es: 0.9803063457330415
El recall es: 0.9931479242241031
El puntaje F1 es: 0.9866853538892781
El coeficiente de correlación de Matthews es: 0.9719882568698346 



Predicho,Activo,Desercion
Observado,,
Activo,0.98,0.02
Desercion,0.01,0.99


In [30]:
resultados = pd.read_excel('data/Resultados.xlsx')
resultados = resultados['NUMEROTARJETA1'].values.tolist()
len(resultados)

55976

In [31]:
test = data['numerotarjeta1'].values.tolist()
len(test)

126541

In [32]:
coinciden = [i for i in resultados if i in test]
len(coinciden)

55939

In [33]:
test = data.set_index('numerotarjeta1')
test = test.loc[coinciden, :]
test['numerotarjeta1'] = test.index
test.reset_index(drop=True, inplace=True)
test.head()

,franquicia,tipo_tarjeta,cupo_global,disponible,total_mes_en_mora,altura_de_mora,aumento_cupo,SUM(trx.valor_transaccion),SUM(trx.numero_cuotas_diferidas),STD(trx.valor_transaccion),STD(trx.numero_cuotas_diferidas),MAX(trx.valor_transaccion),MAX(trx.numero_cuotas_diferidas),SKEW(trx.valor_transaccion),SKEW(trx.numero_cuotas_diferidas),MIN(trx.valor_transaccion),MIN(trx.numero_cuotas_diferidas),MEAN(trx.valor_transaccion),MEAN(trx.numero_cuotas_diferidas),COUNT(trx),NUM_UNIQUE(trx.codigo_transaccion),MODE(trx.codigo_transaccion),DAY(fecha_de_emision),YEAR(fecha_de_emision),MONTH(fecha_de_emision),WEEKDAY(fecha_de_emision),NUM_UNIQUE(trx.DAY(fecha_de_transaccion)),NUM_UNIQUE(trx.MONTH(fecha_de_transaccion)),NUM_UNIQUE(trx.WEEKDAY(fecha_de_transaccion)),NUM_UNIQUE(trx.YEAR(fecha_de_transaccion)),MODE(trx.DAY(fecha_de_transaccion)),MODE(trx.MONTH(fecha_de_transaccion)),MODE(trx.WEEKDAY(fecha_de_transaccion)),MODE(trx.YEAR(fecha_de_transaccion)),numerotarjeta1
0,amex,azul,4000000.00,1309046.44,7,0.00,1,10396826.00,153.00,216847.33,1.83,1600100.00,8.00,4.97,1.26,20000.00,0.00,155176.51,2.28,67.00,3.00,6,31,2002,10,3,26.00,10.00,7.00,1.00,15.00,10.00,2.00,2015.00,8
1,amex,azul,2400000.00,2400000.00,2,0.00,1,283413.24,36.00,20814.13,7.46,40460.00,24.00,-0.39,2.56,10.00,0.00,23617.77,3.00,12.00,3.00,4,12,2003,8,1,5.00,8.00,6.00,1.00,4.00,2.00,2.00,2015.00,12
2,amex,azul,1900000.00,113626.29,0,0.00,0,4788189.71,180.00,386295.18,15.99,1786373.71,36.00,3.73,1.25,17340.00,0.00,239409.49,9.00,20.00,3.00,4,5,2012,6,1,14.00,8.00,5.00,1.00,2.00,3.00,4.00,2015.00,14
3,amex,azul,1300000.00,1300000.00,0,0.00,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,3,2014,12,2,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,19
4,amex,azul,3900000.00,2669408.18,5,0.00,1,3029360.00,3.00,207918.12,0.77,700000.00,3.00,1.71,3.87,17340.00,0.00,201957.33,0.20,15.00,3.00,4,4,2008,1,4,7.00,10.00,5.00,1.00,14.00,2.00,1.00,2015.00,23


In [34]:
test['Email'] = 'stivenlopezg2@gmail.com'
prediccion = pd.DataFrame(np.round(gb.predict_proba(test), 2), columns=['0', 'Pred'])
test = pd.concat([test, prediccion], axis=1)
test = test.loc[:, ['Email', 'numerotarjeta1', 'Pred']]
test.head().append(test.tail())

,Email,numerotarjeta1,Pred
0,stivenlopezg2@gmail.com,8,0.00
1,stivenlopezg2@gmail.com,12,0.00
2,stivenlopezg2@gmail.com,14,0.00
3,stivenlopezg2@gmail.com,19,0.07
4,stivenlopezg2@gmail.com,23,0.00
55934,stivenlopezg2@gmail.com,129828,0.00
55935,stivenlopezg2@gmail.com,129829,0.00
55936,stivenlopezg2@gmail.com,129830,0.00
55937,stivenlopezg2@gmail.com,129831,0.00
55938,stivenlopezg2@gmail.com,129832,0.00


In [36]:
no_coinciden = [i for i in resultados if i not in coinciden]
len(no_coinciden)

37

In [40]:
aux = pd.DataFrame({
    'numerotarjeta1': no_coinciden
})
aux['Email'] = 'stivenlopezg2@gmail.com'
aux['Pred'] = 0.00

In [44]:
test = pd.concat([test, aux], axis=0)
test.columns = ['email', 'NUMEROTARJETA1', 'Pred']
test.to_excel('Resultados.xlsx', index=False)